In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\smryn\AppData\Local\Programs\Python\Python312\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY = "6c1151d2-c0a2-4d13-a8d8-6fbb55d4c1ad"
PINECONE_API_ENV = "gcp-starter"

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("data/")

In [7]:
extracted_data

[Document(page_content='COMMON CLINICAL CONDITIONS AND MINOR AILMENTS', metadata={'source': 'data\\common_clinical_conditions_and_minor_ailments.pdf', 'page': 0}),
 Document(page_content='COMMON CLINICAL CONDITIONS AND MINOR AILMENTS1CONTENTS\nACKNOWLEDGEMENTS 3\n1 ABOUT THIS PACKAGE 4\n2 GASTRO-INTESTINAL SYSTEM  9\n2.1  D yspepsia  1 0 \n2.2 \n G\nastro-oesophageal reflux disease (GORD)\n 1\n4 \n2.3 \n C\nolic\n 1\n7 \n2.4 \n Con\nstipation  2\n0 \n2.5 \n D\niarrhoea\n 2\n5 \n2.6 \n I\nrritable bowel syndrome\n 3\n0 \n2.7 \n H\naemorrhoids (piles)\n 3\n2 \nCase studies  3\n5 \nSuggested responses to gastro-intestinal activities\n 3\n6\n3 RESPIRATORY SYSTEM  39\n3.1  C ough  4 0 \n3.2 \n C\nold\n 4\n6 \n3.3 \n H\nayfever (seasonal allergic rhinitis)\n 5\n0 \nCase studies  53 \nS\nuggested responses to respiratory activities\n 5\n4\n4 CENTRAL NERVOUS SYSTEM  57\n4.1  P ain relief  5 8 \n4.2 \n T\neething in children\n 6\n1 \n4.3 \n M\nusculoskeletal pain – strains, sprains and bruises\

In [8]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 785


In [10]:
text_chunks

[Document(page_content='COMMON CLINICAL CONDITIONS AND MINOR AILMENTS', metadata={'source': 'data\\common_clinical_conditions_and_minor_ailments.pdf', 'page': 0}),
 Document(page_content='COMMON CLINICAL CONDITIONS AND MINOR AILMENTS1CONTENTS\nACKNOWLEDGEMENTS 3\n1 ABOUT THIS PACKAGE 4\n2 GASTRO-INTESTINAL SYSTEM  9\n2.1  D yspepsia  1 0 \n2.2 \n G\nastro-oesophageal reflux disease (GORD)\n 1\n4 \n2.3 \n C\nolic\n 1\n7 \n2.4 \n Con\nstipation  2\n0 \n2.5 \n D\niarrhoea\n 2\n5 \n2.6 \n I\nrritable bowel syndrome\n 3\n0 \n2.7 \n H\naemorrhoids (piles)\n 3\n2 \nCase studies  3\n5 \nSuggested responses to gastro-intestinal activities\n 3\n6\n3 RESPIRATORY SYSTEM  39\n3.1  C ough  4 0 \n3.2 \n C\nold\n 4\n6 \n3.3 \n H', metadata={'source': 'data\\common_clinical_conditions_and_minor_ailments.pdf', 'page': 1}),
 Document(page_content='old\n 4\n6 \n3.3 \n H\nayfever (seasonal allergic rhinitis)\n 5\n0 \nCase studies  53 \nS\nuggested responses to respiratory activities\n 5\n4\n4 CENTRAL NERVO

In [11]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

c:\Users\smryn\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Length 384


In [17]:
# query_result

In [15]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-question-answering"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [16]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What is Chicken Pox"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='APPENDIX 3: A SUMMARY OF CHILDHOOD INFECTIONS\nChickenpox: Chickenpox is caused by a herpes virus, the varicella-zoster virus \n(VZV). It remains a common childhood infection. Chickenpox is highly infectious \nfrom person-to-person contact. It has a relatively long incubation period of \n10 to 21 days and the rash is usually preceded by a day or two of fever. The \nrash is quite distinctive, with crops of small, raised, red spots that develop into'), Document(page_content='airborne droplets & direct contact of weeping lesions & contaminated linens. Mask patient. Provider should avoid contact if they’ve never had chicken pox.  Vaccination now available (1995) and part of childhood immunizations.  Pt isolated until all lesions crusted over and dry. \nCommon cold'), Document(page_content='Shingles \n(varicella- zoster virus)  Second outbreak of the chicken pox virus. Localized manifestation of vesicle with red base on skin areas.  They follow a nerve tract m

In [17]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [18]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [21]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [22]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [23]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\smryn\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  Chickenpox is a common childhood infection caused by the varicella-zoster virus (VZV). It is highly infectious through person-to-person contact, has a relatively long incubation period of 10 to 21 days, and is characterized by crops of small, raised, red spots that develop into blisters.
